In [9]:
from database import database as db
from pandas import Timestamp

In [11]:
current_date = Timestamp('2025-03-21')

In [25]:
query = """
SELECT DISTINCT ON (supplement_id) i.*, d.dosage_per_day
FROM fact.inventory i
JOIN dim.dosage_data d ON i.supplement_id = d.supplement_id
ORDER BY supplement_id, last_updated DESC;
"""

latest_inventory = db.execute_query(query)


In [26]:
latest_inventory

,id,user_id,supplement_id,qty_remaining,last_updated,dosage_per_day
0,1,1,1,60,2025-03-20,2.00
1,2,1,2,90,2025-03-20,2.00
2,3,1,3,60,2025-03-20,2.00
3,4,1,4,30,2025-03-20,1.00
4,5,1,5,120,2025-03-20,1.00
5,6,1,6,60,2025-03-20,1.00
6,7,1,7,60,2025-03-20,1.00


In [27]:
unique_dates = latest_inventory['last_updated'].dropna().unique()

if len(unique_dates) == 1 and unique_dates[0] < current_date:
    print("Calculations for all supplements to follow same dates logic")

    # Calculate daily dosage for each supplement
    latest_inventory['days_since_update'] = (current_date - unique_dates[0]).days
    latest_inventory['qty_use_since_update'] = latest_inventory['days_since_update'] * latest_inventory['dosage_per_day']
    latest_inventory['qty_remaining'] = latest_inventory['qty_remaining'] - latest_inventory['qty_use_since_update']
    latest_inventory = latest_inventory[['user_id', 'supplement_id', 'qty_remaining', 'last_updated']]
    latest_inventory['last_updated'] = current_date

else:
    print("Different calculations for different supplements")


Calculations for all supplements to follow same dates logic


/var/folders/33/494vt_8j5bq4ddhgt8dtv8s80000gn/T/ipykernel_8118/1442332479.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_inventory['last_updated'] = current_date


In [28]:
latest_inventory

,user_id,supplement_id,qty_remaining,last_updated
0,1,1,58.00,2025-03-21
1,1,2,88.00,2025-03-21
2,1,3,58.00,2025-03-21
3,1,4,29.00,2025-03-21
4,1,5,119.00,2025-03-21
5,1,6,59.00,2025-03-21
6,1,7,59.00,2025-03-21


In [3]:
current_date = Timestamp('2025-03-21')
def update_inventory(current_date):


    query = """
    SELECT DISTINCT ON (supplement_id) i.*, d.dosage_per_day
    FROM fact.inventory i
    JOIN dim.dosage_data d ON i.supplement_id = d.supplement_id
    ORDER BY supplement_id, last_updated DESC;
    """

    latest_inventory = db.execute_query(query)

    unique_dates = latest_inventory['last_updated'].dropna().unique()

    if len(unique_dates) == 1 and unique_dates[0] < current_date:
        print("Calculations for all supplements to follow same dates logic")

        # Calculate daily dosage for each supplement
        latest_inventory['days_since_update'] = (current_date - unique_dates[0]).days
        latest_inventory['qty_use_since_update'] = latest_inventory['days_since_update'] * latest_inventory['dosage_per_day']
        latest_inventory['qty_remaining'] = latest_inventory['qty_remaining'] - latest_inventory['qty_use_since_update']
        latest_inventory = latest_inventory[['user_id', 'supplement_id', 'qty_remaining', 'last_updated']]
        latest_inventory['last_updated'] = current_date

    else:
        print("Different calculations for different supplements")

,id,supplement_id,qty_remaining,last_updated
0,1,1,60,2025-03-20
1,2,2,90,2025-03-20
2,3,3,60,2025-03-20
3,4,4,30,2025-03-20
4,5,5,120,2025-03-20
5,6,6,60,2025-03-20
6,7,7,60,2025-03-20
